In [ ]:
!pip install llama-index-llms-bedrock

In [ ]:
!pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api -U -q

In [ ]:
from llama_index.llms.bedrock import Bedrock

llm = Bedrock(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    aws_access_key_id="Aws access key",
    aws_secret_access_key="Aws secret key",
    region_name="Region",
)


In [ ]:
from datasets import load_dataset
from pathlib import Path

from datasets import load_dataset

#ds = load_dataset("neural-bridge/rag-dataset-1200")
dataset = load_dataset(path="neural-bridge/rag-dataset-1200", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, context in enumerate(dataset):
    with open(Path("data") / f"context_{i}.txt", "w") as f:
        f.write(context["context"])

In [ ]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
len(documents)

5960

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
#from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
        #HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ]
)

# run the pipeline sync or async
nodes = await pipeline.arun(documents=documents[:100])
nodes

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./neural_bridge")
chroma_collection = db.get_or_create_collection(name="neural")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(documents=documents[:100])
len(nodes)

In [ ]:
from llama_index.core import VectorStoreIndex
#from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)

In [ ]:
#from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
import nest_asyncio

nest_asyncio.apply()  # This is needed to run the query engine
#llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)


In [ ]:
response = query_engine.query(
    "Who found the answer to a search query collar george herbert essay?"
)
print(response)

According to the context information provided, Francisco Rogers found the answer to the search query "collar george herbert essay".
